In [11]:
import numpy as np
import pandas as pd
import csv
import pickle
from joblib import dump, load
from sklearn import utils
from sklearn import preprocessing
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from gplearn.genetic import SymbolicTransformer, SymbolicRegressor

import datetime
import time

# normalization 

In [62]:
def scaling(x):
    sc = StandardScaler()
    x = sc.fit_transform(x)
    return x

# read data

In [64]:
rawData = pd.read_excel('list-FUC.xlsx')
# 将原始数据分割为两部分
inputData = rawData.iloc[:,1:]
inputData = inputData.drop(['GH'],axis=1)
inputData = scaling(inputData)
outputData = rawData.iloc[:,1]

# initial parameter setting

In [65]:
best_fitness = float('-inf')
best_pc = 0
best_pcoef = 0
best_ps = 0
best_features = []
best_gp = []
function_set = ['add', 'sub', 'mul', 'div', 'abs', 'sqrt']
pc = np.arange(0.6, 0.91, 0.1)
pcoef =  np.arange(0.001, 0.006, 0.001)
pc_grid, pcoef_grid = np.meshgrid(pc, pcoef)
pc_grid = np.array(pc_grid).flatten()
pcoef_grid = np.array(pcoef_grid).flatten()
dimNum = len(pc_grid)
count = 0

# hyperparameter optimization

In [67]:
for i in range(dimNum):
    pc0 = pc_grid[i]
    pcoef0 = pcoef_grid[i]
    ps_max = 1-pc0; ps_min = 0.92-pc0
    ps = np.arange(ps_min, ps_max, 0.01)
    ps = ps/3
    dimNum_ps = len(ps)
    for j in range(dimNum_ps):
        ps0 = ps[j]
        ph0 = ps0
        pp0 = 0.999999999-pc0-ph0-ps0
        gp = SymbolicTransformer(population_size=5000,  function_set=function_set, generations=20, n_components=5,
                        p_crossover=pc0, p_subtree_mutation=ps0,parsimony_coefficient=pcoef0, p_hoist_mutation=ph0, p_point_mutation=pp0,
                         const_range=(-1.0, 1.0), metric='pearson',tournament_size=20, verbose=1)
        print('********************************Round %d********************************' %(count+1))
        print('----------The round of pc=%.6f ps=%.6f pcoef=%.6f pp=%.6f ph=%.6f-----------'
              %(pc0, ps0, pcoef0, pp0, ph0))
        gp.fit(inputData, outputData)
        fitness = gp.run_details_['best_fitness'][-1]
        print('The round of fitness=%.6f\n'%fitness)
        count = count+1
        if fitness > best_fitness:
            best_pc = pc0
            best_pcoef = pcoef0
            best_ps = ps0
            best_features = gp.transform(inputData)
            best_fitness = fitness
            best_gp = gp

********************************第1轮********************************
-----------该轮pc=0.600000 ps=0.106667 pcoef=0.001000 pp=0.186667 ph=0.106667-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    12.93         0.157489        9         0.579354              N/A     54.83s
   1     4.38         0.372995        8          0.70223              N/A     53.92s
   2     4.65         0.457894       16         0.782423              N/A     51.94s
   3     9.09         0.499348       21         0.806468              N/A     51.51s
   4    12.29         0.566819       24         0.824819              N/A     47.39s
   5    17.13         0.615534       31          0.84121              N/A     46.08s
   6    20.35         0.650959       52         0.861582              N/A     42.71s
   7 

  17    15.34          0.71958       16         0.861925              N/A      6.55s
  18    15.43         0.718676       12         0.866514              N/A      3.28s
  19    15.35         0.719783       21         0.874611              N/A      0.00s
该轮fitness=0.874611

********************************第5轮********************************
-----------该轮pc=0.600000 ps=0.120000 pcoef=0.001000 pp=0.160000 ph=0.120000-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    12.56         0.158213        5         0.696766              N/A     54.30s
   1     4.51         0.375631        5         0.708945              N/A     54.56s
   2     4.65         0.460408       12         0.769509              N/A     51.98s
   3     5.97         0.498055       11         0.778125              

  14    13.99         0.711931       32            0.879              N/A     16.29s
  15    15.44         0.717889       32            0.879              N/A     13.19s
  16    16.35         0.723442       33            0.879              N/A     10.19s
  17    16.80         0.727788       34         0.883053              N/A      6.61s
  18    18.23         0.743082       41         0.886245              N/A      3.33s
  19    19.12         0.761666       25         0.884486              N/A      0.00s
该轮fitness=0.884486

********************************第9轮********************************
-----------该轮pc=0.700000 ps=0.073333 pcoef=0.001000 pp=0.153333 ph=0.073333-----------
    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    13.12         0.155515        5         0.653613              

  11    20.30         0.716298       30         0.882997              N/A     27.10s
  12    21.30         0.722727       38         0.887979              N/A     23.76s
  13    23.31         0.739649       38          0.88902              N/A     21.52s
  14    23.96         0.744115       38          0.88902              N/A     17.69s
  15    24.56         0.748011       38         0.888431              N/A     14.24s
  16    25.51         0.757778       30         0.890449              N/A     11.03s
  17    26.95         0.764293       30         0.890449              N/A      7.12s
  18    27.55         0.768566       35         0.894472              N/A      3.58s
  19    26.08         0.758865       45         0.900142              N/A      0.00s
该轮fitness=0.900142

********************************第13轮********************************
-----------该轮pc=0.700000 ps=0.086667 pcoef=0.001000 pp=0.126667 ph=0.086667-----------
    |   Population Average    |             Best Individual

   8    21.08         0.673798       43         0.864885              N/A     41.37s
   9    23.03         0.686608       28         0.875587              N/A     37.72s
  10    25.58         0.711982       32         0.889156              N/A     33.19s
  11    27.85         0.730587       53         0.891399              N/A     30.71s
  12    29.86         0.752816       63         0.901227              N/A     25.01s
  13    31.34         0.759068       63         0.906966              N/A     21.70s
  14    32.72         0.763979       95         0.908786              N/A     17.66s
  15    33.49         0.775947       56         0.912401              N/A     14.28s
  16    33.62         0.780074       59         0.912547              N/A     12.02s
  17    34.09         0.784081       44         0.913815              N/A      7.39s
  18    34.11         0.784042       78         0.917771              N/A      3.59s
  19    33.76         0.785904       61         0.918517         

...

   0    12.60         0.156696        3         0.655103              N/A     53.81s
   1     3.54          0.37466        9         0.748363              N/A     55.22s
   2     3.85         0.471563       10         0.749849              N/A     51.92s
   3     5.87          0.52985       22         0.808274              N/A     52.78s
   4     7.88         0.579086       22         0.808274              N/A     50.22s
   5     9.44           0.6057       23         0.839819              N/A     45.89s
   6    10.53           0.6173       30         0.854953              N/A     42.77s
   7    11.31         0.635452       24         0.867342              N/A     41.20s
   8    11.48         0.642487       24         0.867342              N/A     36.74s
   9    12.36         0.650047       21         0.863757              N/A     32.98s
  10    13.57         0.661178       28         0.864403              N/A     30.66s
  11    13.89         0.667381       23         0.871944         

   0    13.51         0.156204        3         0.645181              N/A     57.59s
   1     3.70         0.378676        5         0.695826              N/A     57.76s
   2     3.52         0.480965        7         0.748168              N/A     54.45s
   3     5.13         0.525692        7         0.777391              N/A     49.47s
   4     5.82         0.565639        7         0.818828              N/A     47.03s
   5     7.17         0.606637       15         0.835324              N/A     43.14s
   6     8.56         0.666126       17         0.840257              N/A     40.58s
   7     7.58         0.665639       24         0.851299              N/A     38.28s
   8     7.02         0.659363       15         0.837951              N/A     34.29s
   9     6.96         0.652711        7         0.818828              N/A     31.14s
  10     7.00         0.659963        7         0.818828              N/A     29.01s
  11     6.92         0.652266        7         0.818828         

   0    12.82          0.15646        6         0.667212              N/A     55.08s
   1     3.68         0.381632        5         0.710916              N/A     56.62s
   2     3.40         0.477206        8         0.739698              N/A     53.07s
   3     5.38         0.517306        7         0.799415              N/A     51.71s
   4     6.47         0.553868        9          0.82168              N/A     48.93s
   5     8.37         0.607448       26         0.855712              N/A     45.06s
   6     9.16          0.65913       12         0.846595              N/A     41.85s
   7     9.00         0.669372       12         0.846595              N/A     39.42s
   8     8.53         0.658246       12         0.846595              N/A     35.59s
   9     8.13         0.657234       13         0.846722              N/A     32.10s
  10     9.57         0.673693       12         0.846722              N/A     29.79s
  11    11.01         0.700463       12         0.846722         

   0    12.93         0.157121        3         0.599685              N/A     55.35s
   1     3.58         0.386651        5         0.712034              N/A     55.84s
   2     3.43           0.4864       16         0.748871              N/A     53.34s
   3     6.02         0.507455       14          0.79392              N/A     52.52s
   4     7.51         0.556363       10         0.802311              N/A     47.91s
   5     8.76         0.604605       19         0.822879              N/A     45.08s
   6    10.77         0.639285       27         0.832998              N/A     43.33s
   7    11.01         0.658545       26         0.835217              N/A     39.28s
   8     9.68         0.650319       25         0.845288              N/A     35.79s
   9     8.08         0.627966       20         0.847584              N/A     33.00s
  10     8.79          0.62859       19         0.845386              N/A     29.05s
  11    10.05         0.640086       18         0.853765         

   0    12.53         0.156879        4         0.693563              N/A     55.39s
   1     3.49         0.384967        5         0.710039              N/A     56.94s
   2     3.23         0.483633        5         0.729008              N/A     54.81s
   3     4.77         0.516544        7          0.81618              N/A     50.81s
   4     5.32         0.546337       15         0.823263              N/A     48.50s
   5     7.34         0.591458       12         0.839299              N/A     44.84s
   6     8.48         0.643705       13         0.843513              N/A     42.34s
   7     8.99          0.66578       19         0.843986              N/A     39.92s
   8     7.81         0.659135       20         0.843954              N/A     35.33s
   9     7.10         0.655369       13         0.845929              N/A     32.15s
  10     6.99         0.652692       17         0.848827              N/A     29.79s
  11     7.12         0.655464       17         0.848827         

# output result

In [68]:
print('The best fitness=%.3f;The best pc=%.6f;The best pcoef=%.6f;The best ps=%.6f'%(best_fitness, best_pc, best_pcoef, best_ps))
print('The best features: %s'%best_features)
print('The best formula: %s'%best_gp)

最佳fitness=0.925;最佳pc=0.600000;最佳pcoef=0.001000;最佳ps=0.110000
最佳特征值为[[ 0.47959614  2.94682729  0.09600125 ...  1.72744832  1.60204665
   2.64138529]
 [-0.3689774   0.65722024  0.07135096 ...  0.6531627  -0.18819786
   0.60978843]
 [ 2.008192    4.63703249  0.63682479 ...  2.0672083   3.19001769
   0.83541056]
 ...
 [16.46490323 24.0958256  13.8733368  ... 25.98780635 10.64154617
  25.24252128]
 [10.68650752 10.83784775  7.85246469 ... 14.13542679  6.08647523
  12.87938423]
 [ 8.23857238 12.8245152   6.45562675 ... 12.79267001  5.86544908
  11.70031293]]
最佳公式为[add(add(add(abs(X77), mul(sqrt(X15), abs(mul(sqrt(X15), abs(add(X1, abs(add(mul(sqrt(X125), abs(X125)), mul(X125, X28))))))))), sub(add(abs(mul(X205, X90)), add(sub(mul(X1, X192), X56), mul(add(abs(X77), sub(X185, X56)), mul(X1, X192)))), X238)), mul(X230, X192)),
 add(add(abs(X77), mul(sqrt(X15), abs(add(X1, abs(abs(add(mul(sqrt(X125), abs(X125)), mul(X125, X28)))))))), sub(add(add(add(abs(X92), mul(X1, X192)), sub(add(mul(add(sqr

# saving model and data

In [70]:
path = "_5D"
data = pd.DataFrame(best_features)
data.to_csv("list_Gpfeature"+path+".csv",index=False,sep=',')
demension = path.split('_')[1]
pickle.dump(best_gp, open(demension+"\\gp"+path+".pickle.dat", "wb"))
data = pd.DataFrame(best_features)
data.to_csv(demension+"list_Gpfeature"+path+".csv",index=False,sep=',')
# best_gp2 = pickle.load(open(demension+"\\gp"+path+".pickle.dat", "rb"))
# dump(best_fitness, demension+"\\best_fitness"+path+".pkl")
# dump(best_pc, demension+"\\best_pc"+path+".pkl")
# dump(best_pcoef, demension+"\\best_pcoef"+path+".pkl")
# dump(best_ps, demension+"\\best_ps"+path+".pkl")
# dump(best_features, demension+"\\best_features"+path+".pkl")
# best_fitness2 = load(demension+"\\best_fitness"+path+".pkl")
# best_pc2 = load(demension+"\\best_pc"+path+".pkl")
# best_pcoef2 = load(demension+"\\best_pcoef"+path+".pkl")
# best_ps2 = load(demension+"\\best_ps"+path+".pkl")
# best_features2 = load(demension+"\\best_features"+path+".pkl")

['D:\\Code\\师妹\\zjnpaper_II\\图4\\图4-b\\best_features_7D.pkl']